In [3]:
import numpy as np
import tensorflow as tf
from tensorflow.keras.models import Sequential
from tensorflow.keras.layers import SimpleRNN, Dense, Embedding
from tensorflow.keras.preprocessing.text import Tokenizer
from tensorflow.keras.preprocessing.sequence import pad_sequences

# Input sentence
sentence = "hello how are you doing today"

# Tokenize sentence
tokenizer = Tokenizer()
tokenizer.fit_on_texts([sentence])
word_index = tokenizer.word_index
print(word_index)
index_word = {v: k for k, v in word_index.items()}



{'hello': 1, 'how': 2, 'are': 3, 'you': 4, 'doing': 5, 'today': 6}


In [8]:
# Create input-output pairs: e.g. ["hello how"] -> "are"
tokens = tokenizer.texts_to_sequences([sentence])[0]
print(tokens)
X_sequences = []
y = []
for i in range(1, len(tokens)):
    X_sequences.append(tokens[:i])
    y.append(tokens[i])

# Pad sequences
max_len = max([len(x) for x in X_sequences])
X_padded = pad_sequences(X_sequences, maxlen=max_len)
y = tf.keras.utils.to_categorical(y, num_classes=len(word_index) + 1)




[1, 2, 3, 4, 5, 6]
[[0 0 0 0 1]
 [0 0 0 1 2]
 [0 0 1 2 3]
 [0 1 2 3 4]
 [1 2 3 4 5]]


In [ ]:
# Build model
model = Sequential()
model.add(Embedding(input_dim=len(word_index) + 1, output_dim=10, input_length=max_len))
model.add(SimpleRNN(64))
model.add(Dense(len(word_index) + 1, activation='softmax'))

# Compile and train
model.compile(loss='categorical_crossentropy', optimizer='adam', metrics=['accuracy'])
model.fit(X_padded, y, epochs=500, verbose=0)

# Prediction function
def predict_next_word(text):
    seq = tokenizer.texts_to_sequences([text])[0]
    seq = pad_sequences([seq], maxlen=max_len)
    pred = model.predict(seq, verbose=0)
    next_index = np.argmax(pred)
    return index_word.get(next_index, "")

# Test
test_text = "how are"
predicted_word = predict_next_word(test_text)
print(f"{test_text} → {predicted_word}")